In [1]:
import gym, pickle, argparse, json, logging
from copy import deepcopy
import ray
from meir import MEIRTrainer

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph
from ray.rllib.agents.ppo.ppo import DEFAULT_CONFIG
from ray.rllib.agents import Trainer
from ray.rllib.evaluation import PolicyEvaluator, SampleBatch, MultiAgentSampleBatchBuilder
from ray.rllib.offline.json_writer import JsonWriter
from ray.rllib.offline.json_reader import JsonReader
from ray.rllib.evaluation.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.evaluation.metrics import collect_metrics
from ray.tune.registry import register_env
from ray.rllib.utils.annotations import override

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder, get_flow_params
logger = logging.getLogger(__name__)
ray.init(ignore_reinit_error=True)

2019-05-27 08:16:01,680	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-27_08-16-01_53957/logs.
2019-05-27 08:16:01,801	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:20213 to respond...
2019-05-27 08:16:01,947	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:33246 to respond...
2019-05-27 08:16:01,950	INFO services.py:804 -- Starting Redis shard with 10.0 GB max memory.
2019-05-27 08:16:02,005	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-27_08-16-01_53957/logs.
2019-05-27 08:16:02,008	INFO services.py:1427 -- Starting the Plasma object store with 18.23 GB memory using /dev/shm.


{'node_ip_address': '10.138.0.2',
 'object_store_address': '/tmp/ray/session_2019-05-27_08-16-01_53957/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-27_08-16-01_53957/sockets/raylet',
 'redis_address': '10.138.0.2:20213',
 'webui_url': None}

In [2]:
num_cpus = 23 
num_rollouts = 20
horizon = 750
gae_lambda = 0.97
step_size = 5e-4
num_iter = 10
benchmark_name = "multi_merge"
exp_name = "test_ir"

In [5]:
config = deepcopy(DEFAULT_CONFIG)
config["num_workers"] = min(num_cpus, num_rollouts)
config["train_batch_size"] = horizon * num_rollouts
config["sample_batch_size"] = horizon / 2
config["use_gae"] = True
config["horizon"] = horizon
config["lambda"] = gae_lambda
config["lr"] = step_size
config["vf_clip_param"] = 1e6
config["num_sgd_iter"] = 10
config['clip_actions'] = False  # FIXME(ev) temporary ray bug
config["model"]["fcnet_hiddens"] = [128, 64, 32]
config["observation_filter"] = "NoFilter"
config["entropy_coeff"] = 0.0
config["num_train"] = 1
config["expert_path"] = './expert_sample'


benchmark_name = "multi_merge"
benchmark = __import__(
            "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.buffered_obs_flow_params
flow_params["env"].additional_params["buf_length"] = 1
create_env, env_name = make_create_env(params=flow_params, version=0)
env = create_env()
register_env(env_name, create_env)

default_policy = (PPOPolicyGraph, env.observation_space, env.action_space, {})
policy_graph = {DEFAULT_POLICY_ID: default_policy}
config["multiagent"] = {
        'policy_graphs': policy_graph,
        'policy_mapping_fn': tune.function(lambda agent_id: DEFAULT_POLICY_ID)
    }

In [6]:
agent = MEIRTrainer(config, env_name)

2019-05-27 08:16:51,421	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/opt/conda/envs/flow-latest/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-05-27 08:16:53,012	INFO policy_evaluator.py:728 -- Built policy map: {'default_policy': <ray.rllib.agents.ppo.ppo_policy_graph.PPOPolicyGraph object at 0x7fd66c2a9e48>}
2019-05-27 08:16:53,013	INFO policy_evaluator.py:729 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fd66c2a9b00>}
2019-05-27 08:16:53,015	INFO policy_evaluator.py:343 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7fd66c2b44e0>}
2019-05-27 08:16:53,557	WARNING json_reader.py:52 -- Treating inpu

In [7]:
with open('1buf.wgt', 'rb') as f:
    weights = pickle.load(f)
    weights[DEFAULT_POLICY_ID] = weights.pop('default')
agent.set_weights(weights)

In [10]:
writer = JsonWriter("./expert_sample")

In [8]:
samples = agent.sample(agent.train_batch_size)
samples.count

(pid=55548) 2019-05-27 08:17:13,208	INFO policy_evaluator.py:437 -- Generating sample batch of size 375.0
(pid=55592) Loading configuration... done.
(pid=55592) Success.
(pid=55587) Loading configuration... done.
(pid=55587) Success.
(pid=55579) Loading configuration... done.
(pid=55579) Success.
(pid=55539) Loading configuration... done.
(pid=55539) Success.
(pid=55573) Loading configuration... done.
(pid=55573) Success.
(pid=55553) Loading configuration... done.
(pid=55553) Success.
(pid=55555) Loading configuration... done.
(pid=55555) Success.
(pid=55567) Loading configuration... done.
(pid=55567) Success.
(pid=55540) Loading configuration... done.
(pid=55540) Success.
(pid=55538) Loading configuration... done.
(pid=55538) Success.
(pid=55571) Loading configuration... done.
(pid=55571) Success.
(pid=55572) Loading configuration... done.
(pid=55572) Success.
(pid=55590) Loading configuration... done.
(pid=55590) Success.
(pid=55548) Loading configuration... done.
(pid=55548) Success

(pid=55587) Loading configuration... done.
(pid=55587) Success.
(pid=55573) Loading configuration... done.
(pid=55565) Loading configuration... done.
(pid=55552) Loading configuration... done.
(pid=55552) Success.
(pid=55568) Loading configuration... done.
(pid=55587) Loading configuration... done.
(pid=55555) Loading configuration... done.
(pid=55555) Success.
(pid=55572) Loading configuration... done.
(pid=55572) Success.
(pid=55568) Success.
(pid=55568) Loading configuration... done.
(pid=55589) Loading configuration... done.
(pid=55589) Success.
(pid=55589) Loading configuration... done.
(pid=55552) Loading configuration... done.
(pid=55555) Loading configuration... done.
(pid=55538) Loading configuration... done.
(pid=55538) Success.
(pid=55572) Loading configuration... done.
(pid=55538) Loading configuration... done.
(pid=55556) Loading configuration... done.
(pid=55556) Success.
(pid=55556) Loading configuration... done.
(pid=55548) 2019-05-27 08:17:21,523	INFO policy_evaluator.

15689

In [11]:
writer.write(sample_batch=samples)

2019-05-27 08:17:27,706	INFO json_writer.py:97 -- Writing to new output file <_io.TextIOWrapper name='/headless/rl_project/flow_codes/ModelBased/expert_sample/output-2019-05-27_08-17-27_worker-0_0.json' mode='w' encoding='UTF-8'>


In [12]:
agent.calculate_expected_feature(samples)

array([0.50089616, 0.09269742, 0.17006099, 0.03464941, 0.08645405,
       0.3768997 , 0.83368564, 0.46684912, 0.60568887, 0.4934346 ,
       0.49899188, 0.09164798], dtype=float32)

In [ ]:
reader = JsonReader("./expert_sample")
sample = reader.next()
sample.count

In [ ]:
reader.next().count